In [58]:
# Import libraries 
import numpy as np
import csv
import random
import pprint 
from qiskit import QuantumCircuit, transpile, Aer, assemble
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator
from timeit import default_timer as timer

In [59]:
# Decide the word to guess there will be two levels: beginer and expert 
def beginer():
    with open('3_letter.csv') as f: 
        reader = csv.reader(f)
        chosen_row = random.choice(list(reader))
        print(chosen_row)
    word = ''

    for x in chosen_row:
        word += ''+ x
    return word

def expert():
    with open('5_letters.csv') as f: 
        reader = csv.reader(f)
        chosen_row = random.choice(list(reader))
        print(chosen_row)
    word = ''
    
    for x in chosen_row:
        word += ''+ x
    return word

In [60]:
# Dictionary Map 

dict_map = {"a":'00000',"b":'00001',"c":'00010',"d":'00011',"e":'00100',"f":'00101',"g":'00110',
            "h":'00111',"i":'01000',"j":'01001',"k":'01010',"l":'01011',"m":'01100',"n":'01101',
            "o":'01110',"p":'01111',"q":'10000',"r":'10001',"s":'10010',"t":'10011',"u":'10100',
            "v":'10101',"w":'10110',"x":'10111',"y":'11000',"z":'11001',
           }

# Key value function
def get_key(val):
    for key, value in dict_map.items():
         if val == value:
            return key
 
    return "There is no such key"

alphabet=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]


In [67]:
def machine_try(word, val):
    startma = timer() 
    def initialize_s(qc, qubits):
        # Apply a H-gate to 'qubits' in qc
        for q in qubits:
            qc.h(q)
        return qc

    def oracle(nqubits, pos):
        # Define an NxN identity matrix; N=2^nqubits; nqubits is the number of qubits
        U = np.identity(2**nqubits)
        # Phase flip the (pos,pos) matrix element
        U[pos][pos] = -1
        qc = QuantumCircuit(nqubits)
        qc.unitary(U, range(nqubits))
        neo = qc.to_gate()
        return neo

    def diffuser(nqubits):
        qc = QuantumCircuit(nqubits)
        # Apply transformation |s> -> |00..0> (H-gates)
        for qubit in range(nqubits):
            qc.h(qubit)
        # Apply transformation |00..0> -> |11..1> (X-gates)
        for qubit in range(nqubits):
            qc.x(qubit)
        # Do multi-controlled-Z gate
        qc.h(nqubits-1)
        qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
        qc.h(nqubits-1)
        # Apply transformation |11..1> -> |00..0>
        for qubit in range(nqubits):
            qc.x(qubit)
        # Apply transformation |00..0> -> |s>
        for qubit in range(nqubits):
            qc.h(qubit)
        # We will return the diffuser as a gate
        U_s = qc.to_gate()
        return U_s
    
    n = 5 
    pos = np.array([-1]*len(word))
    keys = np.array(['-'*n]*len(word))
    print("The machine is gonna search for the ", len(word),"letter word")
    print("Each letter takes a value from the 26-letter English aphlabet,")
    print("which is represented by 5 qubits according the to dictionary")
    print("_"*18)
    print("{:<10} {:<20} ".format('Key','Binary'))
    for key, value in dict_map.items():
        print("{:<10} {:<20} ".format(key, value))
    print("_"*18)
    
    print("="*100)
    print("Quantum setup")
    print("We perform Grover's algorithm for each letter")
    print("="*100)
    for i in range(len(word)):
        
        print("Letter:",i)
        print("_"*100)
        pos[i] = alphabet.index(word[i])
    
        grover_circuit = QuantumCircuit(n)
        grover_circuit = initialize_s(grover_circuit, range(n))
        
        print("Step 1: We initialize the 5 qubits into a uniform superposition")
        print(grover_circuit)
        
        grover_circuit.append(oracle(n, pos[i]), range(n))
        print("Step 2: We apply the oracle")
        print(grover_circuit)
        
        grover_circuit.append(diffuser(n), range(n))
        print("Step 3: We apply the diffuser")
        print(grover_circuit)
        
        print("Step 4: We repeat steps 2 and 3 iteratively on the simulator until we find the desired letter")
        grover_circuit.measure_all()
        aer_sim = Aer.get_backend('aer_simulator') 
        transpiled_grover_circuit = transpile(grover_circuit, aer_sim)
        qobj = assemble(transpiled_grover_circuit)
        results = aer_sim.run(qobj).result()
        counts = results.get_counts()
        max_val = list(counts.values())
        max_ke = list(counts.keys())
        q = max_ke[max_val.index(max(max_val))]
        keys[i] = q
        print("The bit string is", q)
        print("The letter", i, "is", get_key(keys[i]))

    print("="*100)
    if (val == 0):
        print("We combine all", len(word),"letters from the simulator to find the word")
        print("The word is :",get_key(keys[0]),get_key(keys[1]),get_key(keys[2]))
        endma = timer()
    else:
        print("We combine all", len(word),"letters from the simulator to find the word")
        print("The word is :",get_key(keys[0]),get_key(keys[1]),get_key(keys[2]),get_key(keys[3]),get_key(keys[4]))
        endma = timer()
        
    return(endma-startma)

In [68]:
# Main code
# First user is trying to guess the word
print("="*100)
val = int(input("Enter difficulty level => 0 (Beginer) or 1 (Expert):"))
print("="*100)
if (val == 0):
    word = beginer()
    answer = input("Guess the three letter word:")
    starthu = timer()
    while True:
        if (answer == word):
            print("Correct Answer: ", word)
            endhu = timer()
            print("Time taken to guess the correct word by human (in sec):", endhu-starthu)
            break;
        else:
            lst = []
            for i in range(3):
                for posi, char in enumerate(word):
                    if(char == answer[i]):
                        lst.append(posi)
                print("The position of letter", answer[i], lst)
                del lst[:]
            answer = input("Try again: ")
            continue   
else:
    word = expert()
    answer = input("Guess the five letter word: ")
    starthu = timer()
    while True:
        if (answer == word):
            print("Correct Answer: ", word)
            endhu = timer()
            print("Time taken to guess the correct word by human (in sec):", endhu-starthu)
            break;
        else:
            lst = []
            for i in range(5):
                for posi, char in enumerate(word):
                    if(char == answer[i]):
                        lst.append(posi)
                print("The position of letter", answer[i], lst)
                del lst[:]
            answer = input("Try again: ")
            print("_"*100)
            continue  
# Now it's machine's turn to guess
print("="*100)
print("Now the machine will search")
print("="*100)
print("Time taken by the machine (in sec):", machine_try(word, val))

Enter difficulty level => 0 (Beginer) or 1 (Expert):1
['a', 'n', 'g', 'e', 'r']
Guess the five letter word: anger
Correct Answer:  anger
Time taken to guess the correct word by human (in sec): 4.760000001624576e-05
Now the machine will search
The machine is gonna search for the  5 letter word
Each letter takes a value from the 26-letter English aphlabet,
which is represented by 5 qubits according the to dictionary
__________________
Key        Binary               
a          00000                
b          00001                
c          00010                
d          00011                
e          00100                
f          00101                
g          00110                
h          00111                
i          01000                
j          01001                
k          01010                
l          01011                
m          01100                
n          01101                
o          01110                
p          01111                
q 